In [1]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Hop teaming table
db = sqlite3.connect('data/hop_team.sqlite')

for chunk in tqdm(pd.read_csv('data/DocGraph_Hop_Teaming_2018.csv', chunksize = 100000)):
    chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]
    chunk = chunk[(chunk['transaction_count'] >= 50) & (chunk['average_day_wait'] < 50)]
    chunk.to_sql('hop', db, if_exists = 'append', index = False)
    
db.close()

0it [00:00, ?it/s]

In [3]:
# NPPES table
db = sqlite3.connect('data/hop_team.sqlite')
for chunk in tqdm(pd.read_csv('data/npidata_pfile_20050523-20220213.csv', chunksize = 100000)):
    for counter in range(1,16):
        taxonomy_switch = chunk['Healthcare Provider Primary Taxonomy Switch_{}'.format(counter)] == 'Y'
        taxonomy_code_check = 'Healthcare Provider Taxonomy Code_{}'.format(counter)
        taxonomy_code = chunk.loc[taxonomy_switch, taxonomy_code_check]
        chunk.loc[taxonomy_switch, 'Healthcare Provider Taxonomy Code'] = taxonomy_code
    chunk = chunk[['NPI',
     'Entity Type Code',
     'Provider Organization Name (Legal Business Name)',
     'Provider Last Name (Legal Name)',
     'Provider First Name',
     'Provider Middle Name',
     'Provider Name Prefix Text',
     'Provider Name Suffix Text',
     'Provider Credential Text',
     'Provider First Line Business Practice Location Address',
     'Provider Second Line Business Practice Location Address',
     'Provider Business Practice Location Address City Name',
     'Provider Business Practice Location Address State Name',
     'Provider Business Practice Location Address Postal Code',
     'Healthcare Provider Taxonomy Code']]
    chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]
    chunk.fillna(0)
    chunk['provider_business_practice_location_address_postal_code'] = chunk['provider_business_practice_location_address_postal_code'].astype(str)
    chunk['provider_business_practice_location_address_postal_code'] = [zip.replace(zip[:], zip[:5]) if len(zip) > 9 else zip for zip in chunk['provider_business_practice_location_address_postal_code']]
    chunk.to_sql('npi', db, if_exists = 'append', index = False)
db.close()

0it [00:00, ?it/s]

In [4]:
# Taxonomy table
db = sqlite3.connect('data/hop_team.sqlite')
for chunk in tqdm(pd.read_csv('data/nucc_taxonomy_220.csv', chunksize = 100000)):
    chunk = chunk[['Code',
                   'Grouping',
                   'Classification']]
    chunk.columns = [x.lower().replace(' ', '_') for x in chunk.columns]
    chunk.to_sql('taxonomy', db, if_exists = 'append', index = False)
db.close()

0it [00:00, ?it/s]

In [5]:
# CBSA-to-ZIP table
db = sqlite3.connect('data/hop_team.sqlite')
cbsa_excel = pd.read_excel('data/ZIP_CBSA_122021.xlsx', dtype=str)
cbsa_excel.to_sql('cbsa', db, if_exists = 'replace', index = False)
db.close()

In [12]:
#Indices for tables
db = sqlite3.connect('data/hop_team.sqlite')
db.execute('CREATE INDEX from_npi ON hop(from_npi)')
db.execute('CREATE INDEX to_npi ON hop(to_npi)')
db.execute('CREATE INDEX provider_business_practice_location_address_postal_code ON npi(provider_business_practice_location_address_postal_code)')
db.execute('CREATE INDEX provider_business_practice_location_address_city_name ON npi(provider_business_practice_location_address_city_name)')
db.execute('CREATE INDEX provider_business_practice_location_address_state_name ON npi(provider_business_practice_location_address_state_name)')


In [13]:
#Check npi table
db = sqlite3.connect('data/hop_team.sqlite')

query = "SELECT * FROM npi"

npi_sqlite = pd.read_sql(query, db)

db.close()

In [14]:
npi_sqlite

,npi,entity_type_code,provider_organization_name_(legal_business_name),provider_last_name_(legal_name),provider_first_name,provider_middle_name,provider_name_prefix_text,provider_name_suffix_text,provider_credential_text,provider_first_line_business_practice_location_address,provider_second_line_business_practice_location_address,provider_business_practice_location_address_city_name,provider_business_practice_location_address_state_name,provider_business_practice_location_address_postal_code,healthcare_provider_taxonomy_code
0,1679576722,1.0,None,WIEBE,DAVID,A,None,None,M.D.,3500 CENTRAL AVE,None,KEARNEY,NE,68847,207X00000X
1,1588667638,1.0,None,PILCHER,WILLIAM,C,DR.,None,MD,1824 KING STREET,SUITE 300,JACKSONVILLE,FL,32204,207RC0000X
2,1497758544,2.0,"CUMBERLAND COUNTY HOSPITAL SYSTEM, INC",None,None,None,None,None,None,3418 VILLAGE DR,None,FAYETTEVILLE,NC,28304,251G00000X
3,1306849450,NaN,None,None,None,None,None,None,None,None,None,None,None,nan,None
4,1215930367,1.0,None,GRESSOT,LAURENT,None,DR.,None,M.D.,17323 RED OAK DR,None,HOUSTON,TX,77090,207RH0003X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7189073,1649925215,1.0,None,MARTIN,ASHLEY,None,None,None,LMT,1703 DELONEY ST UNIT B,None,AUSTIN,TX,787211111,225700000X
7189074,1528713195,1.0,None,HONG,RYAN,None,None,None,PHARMD,2 BERGEN TPKE,None,RIDGEFIELD PARK,NJ,76602390,183500000X
7189075,1437804002,1.0,None,BROWN,CAMERON,DAVID,None,None,None,233 AUGUSTINE DR,None,SPARTANBURG,SC,293066927,390200000X
7189076,1851046437,1.0,None,MOHAMED,WALEED,None,None,None,None,5021 S 13TH ST,None,MILWAUKEE,WI,532213600,1041C0700X


In [ ]:
db = sqlite3.connect('data/hop_team.sqlite')

query = """
SELECT 
    h.from_npi,
    h.to_npi,
    h.patient_count,
    h.transaction_count,
    h.average_day_wait,
    h.std_day_wait,
    n1.entity_type_code AS from_npi_entity_code,
    n2.entity_type_code AS to_npi_entity_code,
    n1.healthcare_provider_taxonomy_code AS from_npi_taxonomy_code,
    n2.healthcare_provider_taxonomy_code AS to_npi_taxonomy_code,
    t1.grouping AS from_npi_taxonomy_grouping,
    t1.classification AS from_npi_taxonomy_classification,
    t2.grouping AS to_npi_taxonomy_grouping,
    t2.classification AS to_npi_taxonomy_classification
FROM hop AS h
INNER JOIN npi AS n1
ON h.from_npi = n1.npi
INNER JOIN npi AS n2
ON h.to_npi = n2.npi
INNER JOIN taxonomy AS t1
ON t1.code = n2.healthcare_provider_taxonomy_code
INNER JOIN taxonomy AS t2
ON t2.code = n2.healthcare_provider_taxonomy_code
WHERE from_npi_entity_code = 1.0
    AND to_npi_entity_code = 2.0
"""
hop_npi_taxonomy_sqlite = pd.read_sql(query, db)

db.close()

In [2]:
db = sqlite3.connect('data/hop_team.sqlite')

query = "SELECT * FROM npi WHERE provider_business_practice_location_address_state_name = 'TN'"

npi_sqlite = pd.read_sql(query, db)

db.close()

In [3]:
npi_sqlite

,npi,entity_type_code,provider_organization_name_(legal_business_name),provider_last_name_(legal_name),provider_first_name,provider_middle_name,provider_name_prefix_text,provider_name_suffix_text,provider_credential_text,provider_first_line_business_practice_location_address,provider_second_line_business_practice_location_address,provider_business_practice_location_address_city_name,provider_business_practice_location_address_state_name,provider_business_practice_location_address_postal_code,healthcare_provider_taxonomy_code
0,1841293891,1.0,None,GIBBS,ELMER,RICKEY,DR.,None,M.D.,49 CLEVELAND ST 310,None,CROSSVILLE,TN,38555,208600000X
1,1659374601,1.0,None,OBERDICK,WENDY,TIPTON,None,None,MD,105 W STONE DR,STE 1F,KINGSPORT,TN,37660,207Q00000X
2,1134122187,1.0,None,RUDNICKE,CHERYL,DENISE,MRS.,None,CRNP,250 25TH AVE N,STE 412,NASHVILLE,TN,37203,363L00000X
3,1003819046,1.0,None,NYLANDER,BARBARA,H,None,None,M.D.,345 23RD AVE N,SUITE 209,NASHVILLE,TN,37203,207VG0400X
4,1750384780,1.0,None,PERRIGIN,JULIE,A,DR.,None,MD,219 CHURCH ST,None,DICKSON,TN,37055,207Q00000X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123505,1821743493,1.0,None,RUSSELL,KILA,None,None,None,None,3818 S LEE HWY,None,MC DONALD,TN,373535876,224P00000X
123506,1184379760,1.0,None,GAYLE,KAYLA,DANIELLE,None,None,FNP,2325 CRESTMOOR RD STE 101,None,NASHVILLE,TN,372152032,363LF0000X
123507,1891440475,1.0,None,JORDY,JEFFREY,LEE,None,None,LCSW,210 30TH AVE N APT 301,None,NASHVILLE,TN,372031583,101YM0800X
123508,1346995917,1.0,None,TAYLOR,KERI,None,None,None,OTR/L,2018 EXETER RD,None,GERMANTOWN,TN,381383945,225X00000X


In [5]:
db = sqlite3.connect('data/hop_team.sqlite')

query = "SELECT * FROM taxonomy"

taxonomy_sqlite = pd.read_sql(query, db)

db.close()

In [6]:
taxonomy_sqlite

,code,grouping,classification
0,193200000X,Group,Multi-Specialty
1,193400000X,Group,Single Specialty
2,207K00000X,Allopathic & Osteopathic Physicians,Allergy & Immunology
3,207KA0200X,Allopathic & Osteopathic Physicians,Allergy & Immunology
4,207KI0005X,Allopathic & Osteopathic Physicians,Allergy & Immunology
...,...,...,...
863,343800000X,Transportation Services,Secured Medical Transport (VAN)
864,344600000X,Transportation Services,Taxi
865,347D00000X,Transportation Services,Train
866,347E00000X,Transportation Services,Transportation Broker


In [7]:
db = sqlite3.connect('data/hop_team.sqlite')

query = """
SELECT to_npi,  
       provider_organization_name_legal_business_name
FROM hop 
INNER JOIN npi
ON hop.to_npi = npi.npi

"""

hop20 = pd.read_sql(query, db)

db.close()

DatabaseError: Execution failed on sql '
SELECT to_npi,  
       provider_organization_name_legal_business_name
FROM hop 
INNER JOIN npi
ON hop.to_npi = npi.npi

': no such column: provider_organization_name_legal_business_name

In [ ]:
hop20

In [ ]:
db = sqlite3.connect('data/hop_team.sqlite')

query = "SELECT * FROM cbsa WHERE cbsa = '34980'"

cbsa_sqlite = pd.read_sql(query, db)

db.close()

In [ ]:
cbsa_sqlite

In [ ]:
#dropping tables
db = sqlite3.connect('data/hop_team.sqlite')
cursor = db.cursor()
cursor.execute('DROP TABLE npi')
print('Table dropped...')
db.commit()
db.close()
